In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
train_df = pd.read_csv("../input/uec2021-exercise-3/train_sales.csv")
test_df = pd.read_csv("../input/uec2021-exercise-3/test_sales.csv")
stores_df = pd.read_csv("../input/uec2021-exercise-3/stores.csv")
features_df = pd.read_csv("../input/uec2021-exercise-3/features.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
stores_df.head()

In [ ]:
features_df.head()

深く考えずにマージする

In [ ]:
train_large_df = pd.merge(pd.merge(train_df, stores_df), features_df)

In [ ]:
train_large_df.shape, train_df.shape

In [ ]:
test_large_df = pd.merge(pd.merge(test_df, stores_df), features_df)

In [ ]:
test_large_df.shape, test_df.shape

レコード数変わってないし大丈夫そう

In [ ]:
train_large_df.head()

In [ ]:
test_large_df.head()

In [ ]:
 train_test_large_df = pd.concat([train_large_df.drop(["Weekly_Sales"], axis=1), test_large_df], axis=0).reset_index(drop=True)

In [ ]:
categorical_columns = [
    'Store', 'Type', 'IsHoliday'
]

train_test_large_df[categorical_columns] = train_test_large_df[categorical_columns].astype('category')
train_test_large_df.head()

- idとDateは除外する（本当はここで頑張って特徴作るべき）  
- ランダムに分割（とてもよくない）
- Storeはカテゴリ変数として扱う（あまり良くはない）

In [ ]:
train_x_df, valid_x_df, train_y, valid_y = train_test_split(train_test_large_df.iloc[:train_df.shape[0], :].drop(["id", "Date"], axis=1), train_df["Weekly_Sales"].values, test_size=0.2, random_state=42)

In [ ]:
dtrain = lgb.Dataset(train_x_df, train_y)
dvalid = lgb.Dataset(valid_x_df, valid_y)

In [ ]:
best_params = {
    'objective': 'regression', 
    'metric': 'rmse',
    'seed': 71, 
    'num_leaves': 31
}
num_boost_round = 1000
early_stopping_rounds = 100

In [ ]:
model = lgb.train(
    best_params, dtrain,
    valid_sets=[dtrain, dvalid],
    valid_names=['train', 'valid'],
    num_boost_round=num_boost_round,
    early_stopping_rounds=early_stopping_rounds, 
    verbose_eval=100
)

In [ ]:
test_pred = model.predict(train_test_large_df.iloc[train_df.shape[0]:, :].drop(["id", "Date"], axis=1))

In [ ]:
sub_df = pd.DataFrame({
    "id": test_df["id"], 
    "pred": test_pred
})
sub_df.head()

In [ ]:
sub_df.to_csv("submission.csv", index=False)

In [ ]:
nan